In [4]:
import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt

from plotly import express as px
import cufflinks
cufflinks.go_offline()

from pprint import pprint
from glob import glob
from tqdm.notebook import tqdm_notebook as tqdm

import sys
sys.path.append('../')

from src.data.gibdd_filter import GibddFilter
# from src.data import normalize_name

import warnings
warnings.simplefilter('ignore')

In [5]:
def preproc_spark_data(df):
    
    df = df.copy()
    df.columns = (df.iloc[2])
    df = df.iloc[3:-2]
    df.set_index('№', inplace=True)
    
    return df

In [6]:
SPARK_PATH = '../data/external/spark_interfax/СПАРК_Выборка_компаний_20200901_1552.xlsx'
GIBDD_PATH = '../data/processed/gibdd.parquet'

In [7]:
df_gibdd = pd.read_parquet(GIBDD_PATH)
df_spark = preproc_spark_data(pd.read_excel(SPARK_PATH))

df_spark = df_spark.dropna(axis=1, how='all')
col_numeric = df_spark.loc[:,'2017, Основные средства , RUB':].columns.tolist() + ['Возраст компании, лет', 'ИДО', 'ИФР', 'ИПД', 'Сумма незавершенных исков в роли ответчика, RUB', 'Сумма исполнительных производств']
for col in col_numeric:
    df_spark[col] = df_spark[col].astype(float)

In [9]:
df_spark.sample(5)

2,Наименование,Регистрационный номер,Краткое наименование,Руководитель - ФИО,Телефон,Электронный адрес,Сайт в сети Интернет,"Возраст компании, лет",Статус,Код налогоплательщика,ИДО,ИФР,ИПД,Сводный индикатор,"Сумма незавершенных исков в роли ответчика, RUB",Сумма исполнительных производств,Размер компании,Важная информация,Мои списки,"2017, Основные средства , RUB","2018, Основные средства , RUB","2019, Основные средства , RUB","2017, Внеоборотные активы, RUB","2018, Внеоборотные активы, RUB","2019, Внеоборотные активы, RUB","2017, Чистые активы, RUB","2018, Чистые активы, RUB","2019, Чистые активы, RUB","2017, Запасы, RUB","2018, Запасы, RUB","2019, Запасы, RUB","2017, Дебиторская задолженность, RUB","2018, Дебиторская задолженность, RUB","2019, Дебиторская задолженность, RUB","2017, Уставный капитал , RUB","2018, Уставный капитал , RUB","2019, Уставный капитал , RUB","2017, Кредиторская задолженность, RUB","2018, Кредиторская задолженность, RUB","2019, Кредиторская задолженность, RUB","2017, Выручка, RUB","2018, Выручка, RUB","2019, Выручка, RUB","2017, Чистая прибыль (убыток), RUB","2018, Чистая прибыль (убыток), RUB","2019, Чистая прибыль (убыток), RUB","2017, EBIT, RUB","2018, EBIT, RUB","2019, EBIT, RUB","2017, Коэффициент соотношения заемных и собственных средств, %","2018, Коэффициент соотношения заемных и собственных средств, %","2019, Коэффициент соотношения заемных и собственных средств, %","2017, Коэффициент маневренности собственных средств, %","2018, Коэффициент маневренности собственных средств, %","2019, Коэффициент маневренности собственных средств, %","2017, Коэффициент обеспеченности собственными оборотными средствами, %","2018, Коэффициент обеспеченности собственными оборотными средствами, %","2019, Коэффициент обеспеченности собственными оборотными средствами, %","2017, Коэффициент текущей ликвидности, %","2018, Коэффициент текущей ликвидности, %","2019, Коэффициент текущей ликвидности, %"
№,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
260,"ДЭП № 33, АО",1103338000370,"АО ""ДЭП № 33""",Айдакин Сергей Павлович,"+7 (343) 3820613, +7 (49233) 22282, +7 (49233)...",dep8@mail.ru,NaN,26.5,Действующая,3338006713,1.0,34.0,NaN,Низкий риск,33644743.0,627914.0,Крупные предприятия,NaN,"""INN"", ""Парк автомобилей""",5.119600e+07,1.581790e+08,6.653090e+08,5.119600e+07,1.581890e+08,6.945210e+08,9.218600e+07,1.738990e+08,1.567180e+08,1.472000e+06,1.068040e+08,9.413400e+07,1.199840e+08,1.520830e+08,3.205670e+08,3.686400e+07,3.686400e+07,3.686400e+07,2.949370e+08,6.177600e+08,6.820920e+08,1.145341e+09,1.256923e+09,3.623196e+09,5.543400e+07,1.017790e+08,34641000.0,67539000.0,1.347430e+08,8.436000e+07,3.2004,4.2260,7.9877,0.4446,0.0903,-3.4317,0.1220,0.0209,-0.7532,1.1389,1.2137,0.7410
121,"БН-СПЕЦТРАНС, ООО",1168901054541,"ООО ""БН-СПЕЦТРАНС""",Ерёменко Николай Валерьевич,"+7 (34936) 28678, +7 (34936) 28679, +7 (951) 9...",n.balovneva@beloil.by,NaN,4.0,Действующая,8911010960,1.0,90.0,NaN,Средний риск,0.0,1000.0,Средние предприятия,NaN,"""INN"", ""Парк автомобилей""",3.229970e+08,3.043530e+08,3.625860e+08,3.310180e+08,3.396690e+08,4.407420e+08,2.396420e+08,7.537100e+07,-1.156150e+08,6.728500e+07,6.710800e+07,8.057100e+07,3.265200e+08,1.524380e+08,1.419960e+08,4.455860e+08,5.025150e+08,5.242080e+08,3.911920e+08,2.309980e+08,2.484430e+08,1.387873e+09,1.260426e+09,1.296705e+09,-2.019940e+08,-2.251920e+08,-216748000.0,-207528000.0,-2.521070e+08,-2.568840e+08,2.1285,6.5131,-7.0380,-0.3813,-3.5066,4.8122,-0.2182,-1.1664,-2.1619,0.9705,0.6710,0.6478
17,"АВТОДОР, АО",1124205005540,"АО ""АВТОДОР""",Лобанов Андрей Анатольевич,"+7 (8342) 210588, +7 (384) 2210588, +7 (384) 2...",avtodor.kemtel@mail.ru,NaN,8.0,Действующая,4205241205,1.0,19.0,84.0,Низкий риск,79280231.0,NaN,Крупные предприятия,NaN,"""INN"", ""Парк автомобилей""",4.708720e+08,5.082820e+08,6.891750e+08,6.096410e+08,5.459130e+08,7.419640e+08,8.309960e+08,4.524460e+08,4.655010e+08,3.330820e+08,5.822000e+08,5.019370e+08,5.056030e+08,2.063600e+08,3.246100e+08,7.67

In [67]:
df_carpark = pd.read_excel('../reports/data/carpark_update_freq.xlsx').set_index('Unnamed: 0')

In [77]:
df_spark['Код налогоплательщика'] = df_spark['Код налогоплательщика'].astype(np.int64)
df_spark.set_index('Код налогоплательщика', inplace=True)

In [89]:
SPARK_COLUMNS = [
    'Наименование',
    'Телефон',
    'Возраст компании, лет',
    'Статус',
    'Сводный индикатор',   
] + df_spark.loc[:, '2017, Дебиторская задолженность, RUB':].columns.tolist()

In [90]:
SPARK_COLUMNS

['Наименование',
 'Телефон',
 'Возраст компании, лет',
 'Статус',
 'Сводный индикатор',
 '2017, Дебиторская задолженность, RUB',
 '2018, Дебиторская задолженность, RUB',
 '2019, Дебиторская задолженность, RUB',
 '2017, Уставный капитал , RUB',
 '2018, Уставный капитал , RUB',
 '2019, Уставный капитал , RUB',
 '2017, Кредиторская задолженность, RUB',
 '2018, Кредиторская задолженность, RUB',
 '2019, Кредиторская задолженность, RUB',
 '2017, Выручка, RUB',
 '2018, Выручка, RUB',
 '2019, Выручка, RUB',
 '2017, Чистая прибыль (убыток), RUB',
 '2018, Чистая прибыль (убыток), RUB',
 '2019, Чистая прибыль (убыток), RUB',
 '2017, EBIT, RUB',
 '2018, EBIT, RUB',
 '2019, EBIT, RUB',
 '2017, Коэффициент соотношения заемных и собственных средств, %',
 '2018, Коэффициент соотношения заемных и собственных средств, %',
 '2019, Коэффициент соотношения заемных и собственных средств, %',
 '2017, Коэффициент маневренности собственных средств, %',
 '2018, Коэффициент маневренности собственных средств, %',

In [99]:
df_carpark = pd.read_excel('../reports/data/carpark_struct_update.xlsx', index_col=[0])

In [100]:
df_carpark_spark = df_carpark.merge(df_spark[SPARK_COLUMNS], left_index=True, right_on='Код налогоплательщика')